# LangChain Expression Language (LCEL)

---

## Introduction

<img src="../../images/lcel.png" alt="LCEL" style="width: 60%; height: auto;"/>
<br/><br/>
<img src="../../images/interface.png" alt="Interface" style="width: 60%; height: auto;"/>

## Advantages of LCEL

Runnables support:
- Async, Batch and Streaming Support
- Fallbacks
- Parallelism
    - LLM calls can be time consuming!
    - Any components that can be run in parallel are!
- Logging is built in

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

deployment_id = "gpt40125"

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = AzureChatOpenAI(deployment_name=deployment_id)
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

"Why don't bears wear socks?\n\nBecause they like to walk bear-foot!"

## More complex chain

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch


vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()

In [7]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [8]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [11]:
chain.invoke({"question": "where did harrison work?"})

'Based on the context provided, Harrison worked at Kensho.'

### Let's look at what `RunnableMap` did

In [12]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [13]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind OpenAI Functions

In [14]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = AzureChatOpenAI(
    deployment_name=deployment_id,
    temperature=0
).bind(functions=functions)

In [16]:
runnable = prompt | model

In [17]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}})

### Let's add another function

In [18]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [19]:
model = AzureChatOpenAI(
    deployment_name=deployment_id,
    temperature=0
).bind(functions=functions)

In [20]:
runnable = prompt | model

In [21]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}})

## Fallbacks

### Let's use newer model where json decode failure can be seen

In [22]:
import json


new_model = AzureChatOpenAI(deployment_name=deployment_id) | StrOutputParser()
failing_chain = model | json.loads

In [23]:
challenge = "write three poems in a json blob, where each poem is a \
json blob of a title, author, and first line, the entire json blob should \
be a list of poems."

In [24]:
new_model.invoke(challenge)

'```json\n[\n  {\n    "title": "Moonlit Night",\n    "author": "Luna M. Sterling",\n    "firstLine": "In the quiet of the moonlit night,"\n  },\n  {\n    "title": "The Last Leaf",\n    "author": "Autumn R. Windham",\n    "firstLine": "A single leaf, brown at the edges,"\n  },\n  {\n    "title": "Ocean\'s Lullaby",\n    "author": "Marina T. Waves",\n    "firstLine": "Whispers of waves caress the shore,"\n  }\n]\n```'

The next cell will fail because the output from `new_model` isn't a valid json.

In [ ]:
failing_chain.invoke(challenge)

### Let's create another chain with an older model that will output json

In [26]:
from langchain.llms import AzureOpenAI
import json

simple_model = AzureOpenAI(
    temperature=0, 
    max_tokens=1000, 
    deployment_name="gpt35_turbo_instruct_0914"
)
passing_chain = simple_model | json.loads

In [27]:
simple_model.invoke(challenge)

'\n\n[\n  {\n    "title": "Autumn Leaves",\n    "author": "Jane Smith",\n    "first_line": "Crisp leaves crunch underfoot"\n  },\n  {\n    "title": "The Ocean\'s Song",\n    "author": "John Doe",\n    "first_line": "Waves crash against the shore"\n  },\n  {\n    "title": "Whispers in the Wind",\n    "author": "Samantha Jones",\n    "first_line": "Soft whispers in the wind"\n  }\n]'

In [28]:
passing_chain.invoke(challenge)

[{'title': 'Autumn Leaves',
  'author': 'Jane Smith',
  'first_line': 'Crisp leaves crunch underfoot'},
 {'title': "The Ocean's Song",
  'author': 'John Doe',
  'first_line': 'Waves crash against the shore'},
 {'title': 'Whispers in the Wind',
  'author': 'Samantha Jones',
  'first_line': 'Soft whispers in the wind'}]

### Let's create the fallback

In [29]:
final_chain = failing_chain.with_fallbacks([passing_chain])

In [30]:
final_chain.invoke(challenge)

[{'title': 'Autumn Leaves',
  'author': 'Jane Smith',
  'first_line': 'Crisp leaves crunch underfoot'},
 {'title': "The Ocean's Song",
  'author': 'John Doe',
  'first_line': 'Waves crash against the shore'},
 {'title': 'Whispers in the Wind',
  'author': 'Samantha Jones',
  'first_line': 'Soft whispers in the wind'}]

## Interface

In [32]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = AzureChatOpenAI(deployment_name=deployment_id)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

### `invoke`

In [34]:
# synchronous method for sending one input
resp = chain.invoke({"topic": "bears"})
print(resp)

Why don't bears wear socks?

Because they like to walk bear-foot!


### `ainvoke`

In [45]:
# asynchronous method for sending one input
response = await chain.ainvoke({"topic": "bears"})
print(response)

Why don't bears wear shoes?

Because they prefer bear feet!


### `batch`

In [36]:
# synchronous method for sending multiple inputs
# parallelizes as much as possible
resp = chain.batch([{"topic": "bears"}, {"topic": "frogs"}])
print(resp)

["Why don't bears wear socks?\n\nBecause they like to walk bear-foot!", 'Why are frogs so happy? \n\nBecause they eat whatever bugs them!']


### `abatch`

In [46]:
# asynchronous method for sending multiple inputs
# parallelizes as much as possible
resp = await chain.abatch([{"topic": "bears"}, {"topic": "frogs"}])
print(resp)

["Why don't bears wear socks?\n\nBecause they like to walk bear-foot!", 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']


### `stream`

In [59]:
# synchronous method for sending one input and streaming the response
for chunk in chain.stream({"topic": "bears"}):
    print(chunk)


Why
 do
 bears
 never
 wear
 socks
?


Because
 they
 like
 to
 walk
 around
 with
 bear
 feet
!



### `astream`

In [60]:
# asynchronous method for sending one input and streaming the response
async for chunk in chain.astream({"topic": "bears"}):
    print(chunk)


Why
 don
't
 bears
 wear
 socks
?


Because
 they
 have
 bear
 feet
!

